In [12]:
import load_data
import matplotlib.pyplot as plt
import numpy as np

## Odometry Dead Reconing

In [46]:
WHEEL_DIAM = 254    # wheel diameter (mm)
BOT_WIDTH = 393.7   # distance from center of left wheel to center of right (mm)

WHEEL_CIRC = np.pi*WHEEL_DIAM    # wheel circumference (mm)
ENC_TICK_LEN = WHEEL_CIRC / 360  # length of one encoder tick (mm)

In [8]:
# Front Right, Front Left,... encoder readings, time
FR_enc, FL_enc, RR_enc, RL_enc, ts = load_data.get_encoder('../data/Encoders20')

In [23]:
R_enc = (FR_enc + RR_enc) / 2  #average value of right encoders
L_enc = (FL_enc + RL_enc) / 2  #average value of left encoders

In [47]:
def get_local_movement(R_enc, L_enc):  # get movement and angular change with bot frame of reference
    R_mm = ENC_TICK_LEN * R_enc  # convert encoder to right wheel movement in mm
    L_mm = ENC_TICK_LEN * L_enc  # convert encoder to left wheel movement in mm
    d_theta = (R_mm - L_mm) / BOT_WIDTH  # find change in angle
    d_i = (R_mm + L_mm) / 2 * np.cos(d_theta)  # motion in i direction
    d_j = (R_mm + L_mm) / 2 * np.sin(d_theta)  # motion in j direction
    return [d_i, d_j, d_theta]

In [45]:
pos = np.zeros((len(FR_enc)+1,2))    # 2D position; assume initial position is (0,0)
angle = np.zeros((len(FR_enc)+1,2))  # orientation; assume initial orientation is 0deg

for i in range(len(FR_enc)):
    d_i, d_j, d_angle = get_local_movement(R_enc[i],L_enc[i])  # local movement
    d_x = (d_i * np.cos(angle[i])) + (d_j * np.sin(angle[i]))  # global x movement
    d_y = (d_i * np.sin(angle[i])) + (d_j * np.cos(angle[i]))  # global y movement
    
    angle[i+1] = angle[i] + d_angle  # update next angle using calculated change
    pos[i+1,0] = pos[i,0] + d_x  # update next x position using calculated change
    pos[i+1,1] = pos[i,1] + d_y  # update next y position using calculated change

(4957, 2)